# Data Science i skyen: "Azure ML SDK"-metoden

## Introduksjon

I denne notatboken skal vi lære hvordan vi bruker Azure ML SDK til å trene, distribuere og bruke en modell gjennom Azure ML.

Forutsetninger:
1. Du har opprettet et Azure ML-arbeidsområde.
2. Du har lastet opp [Heart Failure-datasettet](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) til Azure ML.
3. Du har lastet opp denne notatboken til Azure ML Studio.

De neste stegene er:

1. Opprett et eksperiment i et eksisterende arbeidsområde.
2. Opprett en Compute-klynge.
3. Last inn datasettet.
4. Konfigurer AutoML ved hjelp av AutoMLConfig.
5. Kjør AutoML-eksperimentet.
6. Utforsk resultatene og finn den beste modellen.
7. Registrer den beste modellen.
8. Distribuer den beste modellen.
9. Bruk endepunktet.

## Azure Machine Learning SDK-spesifikke imports


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Initialiser arbeidsområde
Initialiser et arbeidsområdeobjekt fra lagret konfigurasjon. Sørg for at konfigurasjonsfilen er til stede på .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Opprett et Azure ML-eksperiment

La oss opprette et eksperiment kalt 'aml-experiment' i arbeidsområdet vi nettopp initialiserte.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Opprett en beregningsklynge
Du må opprette et [beregningsmål](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) for kjøringen av AutoML.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Data
Sørg for at du har lastet opp datasettet til Azure ML og at nøkkelen har samme navn som datasettet.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Distribuer den beste modellen

Kjør følgende kode for å distribuere den beste modellen. Du kan se statusen for distribusjonen i Azure ML-portalen. Dette steget kan ta noen minutter.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Bruke endepunktet
Du kan legge til inndata i følgende eksempel på inndata.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi tilstreber nøyaktighet, vennligst vær oppmerksom på at automatiske oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for eventuelle misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
